In [ ]:
!unzip -a '/content/drive/My Drive/NLP/input/fake-news.zip'

Archive:  /content/drive/My Drive/NLP/input/fake-news.zip
  inflating: submit.csv              [binary]
  inflating: test.csv                [binary]
  inflating: train.csv               [binary]


In [ ]:
!pip install texthero

In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import gensim.downloader as api
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import nltk
import warnings
from tensorflow.keras.utils import plot_model
from tensorflow.keras import Sequential, layers, losses, callbacks, metrics, Model
from tensorflow.keras.layers import Input, LSTM, Flatten, BatchNormalization, Dense, Activation, Concatenate
import tensorflow.keras.backend as K
import tensorflow_hub as hub
import texthero as hero
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
import operator 
warnings.filterwarnings('ignore')
tqdm.pandas()

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
df.head(2)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0


In [ ]:
text = hero.clean(df['text'])
label = np.array(df['label'])

In [ ]:
elmo = hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                      input_shape=[], dtype = tf.string,
                      trainable=False)

In [ ]:
base_model = Sequential([
                         elmo,
                         layers.Dense(16),
                         layers.Dense(1, activation='sigmoid')
])

In [ ]:
base_model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=True),
              metrics=[tf.metrics.AUC(name='auc'), 'accuracy'])

In [ ]:
base_model.fit(text, label, epochs=3, validation_split=0.2)

Epoch 1/3
455/455 [==============================] - 48s 106ms/step - loss: 0.6537 - auc: 0.7480 - accuracy: 0.6303 - val_loss: 0.6223 - val_auc: 0.8513 - val_accuracy: 0.7377
Epoch 2/3
455/455 [==============================] - 47s 103ms/step - loss: 0.6104 - auc: 0.8663 - accuracy: 0.7612 - val_loss: 0.6059 - val_auc: 0.8718 - val_accuracy: 0.7887
Epoch 3/3
455/455 [==============================] - 47s 103ms/step - loss: 0.5981 - auc: 0.8813 - accuracy: 0.7921 - val_loss: 0.5961 - val_auc: 0.8789 - val_accuracy: 0.7885


In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('./model.hdf5', monitor='val_auc')

In [ ]:
model = Sequential([
                    hub.KerasLayer('https://tfhub.dev/google/universal-sentence-encoder/4',
                      input_shape=[], dtype = tf.string,
                      trainable=True),
                    layers.Reshape((512,1)),
                    layers.LSTM(32),
                    layers.Dropout(0.6),
                    layers.Dense(32),
                    layers.Dropout(0.6),
                    layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.losses.BinaryCrossentropy(from_logits=False),
              metrics=[tf.metrics.AUC(name='auc'), 'accuracy'])

In [ ]:
model.fit(text, label, epochs=5, validation_split=0.2, callbacks=checkpoint)

Epoch 1/5
520/520 [==============================] - 174s 335ms/step - loss: 0.3615 - auc: 0.9203 - accuracy: 0.8274 - val_loss: 0.1264 - val_auc: 0.9905 - val_accuracy: 0.9524
Epoch 2/5
520/520 [==============================] - 173s 332ms/step - loss: 0.0581 - auc: 0.9965 - accuracy: 0.9854 - val_loss: 0.1017 - val_auc: 0.9930 - val_accuracy: 0.9639
Epoch 3/5
520/520 [==============================] - 172s 331ms/step - loss: 0.0146 - auc: 0.9993 - accuracy: 0.9972 - val_loss: 0.3520 - val_auc: 0.9739 - val_accuracy: 0.9591
Epoch 4/5
520/520 [==============================] - 172s 331ms/step - loss: 0.0089 - auc: 0.9996 - accuracy: 0.9984 - val_loss: 0.3718 - val_auc: 0.9699 - val_accuracy: 0.9615
Epoch 5/5
520/520 [==============================] - 173s 333ms/step - loss: 0.0063 - auc: 0.9997 - accuracy: 0.9990 - val_loss: 0.4441 - val_auc: 0.9663 - val_accuracy: 0.9567
